## S3 upload and download

In [1]:
import os
from brtdevkit.util.aws import s3

Internal AWS credentials will be removed from future brt-devkit versions.


In [2]:
s3_client = s3.S3()
BRT_DEVKIT_WRITE_S3_BUCKET = 'mesa-states'
MODEL_TRAINING_S3_KEY_PREFIX = f"{os.environ.get('BRT_ENV', 'prod')}/jupiter/model_training"
PACK_PERCEPTION_S3_KEY_PREFIX = f"{os.environ.get('BRT_ENV', 'prod')}/jupiter/pack_perception"
bucket_name=BRT_DEVKIT_WRITE_S3_BUCKET

In [ ]:
# upload file
# key = f"{MODEL_TRAINING_S3_KEY_PREFIX}/Jupiter_train_v4_53_missing_human_relabeled.tar"
key = f"{PACK_PERCEPTION_S3_KEY_PREFIX}/Jupiter_train_v4_53_heavy_dust_relabeled.tar"
file_path = "/home/bluerivertech/li.yu/data/Jupiter_train_v4_53_heavy_dust_relabeled/Jupiter_train_v4_53_heavy_dust_relabeled.tar"
s3_client.upload_file(
    bucket_name=bucket_name, key=key, file_path=file_path)
# Jupiter_2022_fn_bag1.tar
# Jupiter_2022Jan15Feb15_rock_stops.tar
# Jupiter_train_v4_53_missing_human_relabeled.tar
# Jupiter_train_v4_53_heavy_dust_relabeled.tar

In [4]:
# download file
# run_id = '45_3_6class_ml_pp_seg_lite_cloud_v10_rotate_fix'
# name = 'driveable_terrain_model_val_bestmodel.pth'
# key = f"{MODEL_TRAINING_S3_KEY_PREFIX}/{run_id}/{name}"

key = f"{PACK_PERCEPTION_S3_KEY_PREFIX}/Jupiter_train_v4_53_heavy_dust_relabeled.tar"

target_path = '/data/jupiter/li.yu/data/Jupiter_train_v4_53/Jupiter_train_v4_53_heavy_dust_relabeled/Jupiter_train_v4_53_heavy_dust_relabeled.tar'
s3_client.download_file(bucket_name, key, target_path)

True

## Resave pretrained weights

In [1]:
import torch
from collections import OrderedDict

# d = torch.load('/mnt/sandbox1/ben.cline/data/20240110/fpn_80k_nextvit_small_1n1k6m_pretrained.pth', map_location=lambda storage, loc: storage)
# print(d.keys())
# ori_keys = d['state_dict'].keys()

In [2]:
# supervised pretraining lite12 model
checkpoint_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
model = 'openimages_v7_brtlite12_0211'
checkpoint = 'checkpoint_best'  # checkpoint or checkpoint_best
input_dict = torch.load(os.path.join(checkpoint_dir, model, f'{checkpoint}.pth'), map_location=lambda storage, loc: storage)
output_dict = OrderedDict()
for k, v in input_dict['model'].items():
    output_dict[k] = v
output_dict = {'state_dict': output_dict}
print(output_dict.keys())
torch.save(output_dict, os.path.join(checkpoint_dir, model, f'{checkpoint}_brt_compatible.pth'))

dict_keys(['state_dict'])


In [3]:
# supervised pretraining
checkpoint_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
model = 'openimages_v7_nextvit_small_structuralreparam_0311'
checkpoint = 'checkpoint'  # checkpoint or checkpoint_best
input_dict = torch.load(os.path.join(checkpoint_dir, model, f'{checkpoint}.pth'), map_location=lambda storage, loc: storage)
output_dict = OrderedDict()
for k, v in input_dict['model'].items():
    output_dict['backbone.' + k] = v
output_dict = {'state_dict': output_dict}
print(output_dict.keys())

# check difference between open source weights and OpenImages pretrained weights
# print(set(output_dict['state_dict'].keys()) - set(ori_keys))

torch.save(output_dict, os.path.join(checkpoint_dir, model, f'{checkpoint}_brt_compatible.pth'))

dict_keys(['state_dict'])


In [2]:
# self-supervised pretraining
checkpoint_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
# model = 'brt1m_spark_nextvit_small_0307'
# model = 'openimages_v7_spark_nextvit_small_0226'
model = 'train600k_spark_nextvit_small_0329'
checkpoint = 'nextvit_small_pretrained_timm_style'  # checkpoint or checkpoint_best
input_dict = torch.load(os.path.join(checkpoint_dir, model, f'{checkpoint}.pth'), map_location=lambda storage, loc: storage)
output_dict = OrderedDict()
for k, v in input_dict.items():
    output_dict['backbone.' + k] = v
output_dict = {'state_dict': output_dict}
print(output_dict.keys())

# # check difference between open source weights and OpenImages pretrained weights
# print(set(output_dict['state_dict'].keys()) - set(ori_keys))

torch.save(output_dict, os.path.join(checkpoint_dir, model, f'{checkpoint}_brt_compatible.pth'))

dict_keys(['state_dict'])


## Resave trained weights for kore

In [1]:
import torch
from collections import OrderedDict

In [2]:
checkpoint_dir1 = '/mnt/sandbox1/ben.cline/output/bc_sandbox_2024/'
checkpoint_dir2 = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
# model = 'ds_v8_1_nextvit_small_openimages_with_rev1_train_human_test_using_random_val_mhc_20_epochs_finetune_rev1_lr'
# model = 'ds_v8_1_nextvit_small_openimages_using_random_val_50k_rev1_rev2_lying_down_sitting_100_100'
# model = 'ds_v8_1_nextvit_small_openimages_using_random_val_50k_rev1_rev2_lying_down_sitting_2x_lower_airborne_birds_lower_msl_d'
# model = 'ds_v8_1_nextvit_small_openimages_using_random_val_50k_rev1_rev2_lying_down_sitting_2x_lower_airborne_birds_lower_msl_d_2h'
model = 'all_rev2_rev1_lying_down_sitting_headlights_round_2_25_ep_prod_weights_10_lo_10_tr'
model = '9_2_unofficial_50k_rev1_rev2_lying_down_sitting_headlights_25_ep_prod_weights_10_lo_100_tr_2'
# checkpoint = 'bc_sandbox_2024_val_bestmodel_master_6'  
checkpoint = 'bc_sandbox_2024_val_bestmodel'  
input_dict = torch.load(os.path.join(checkpoint_dir1, model, f'{checkpoint}.pth'), map_location=lambda storage, loc: storage)
print(input_dict.keys())

dict_keys(['epoch', 'state_dict', 'optimizer', 'scheduler'])


In [4]:
output_dict = OrderedDict()
output_dict['epoch'] = input_dict['epoch']
output_dict['state_dict'] = OrderedDict()
for k,v in input_dict['state_dict'].items():
    if k.startswith('scale_'):
        output_dict['state_dict']['head.'+k] = v
    else:
        output_dict['state_dict'][k] = v

os.makedirs(os.path.join(checkpoint_dir2, model), exist_ok=True)
saved_path = os.path.join(checkpoint_dir2, model, f'{checkpoint}_brtkore_compatible.pth')
torch.save(output_dict, saved_path)
print(saved_path)

/data/jupiter/li.yu/exps/driveable_terrain_model/9_2_unofficial_50k_rev1_rev2_lying_down_sitting_headlights_25_ep_prod_weights_10_lo_100_tr_2/bc_sandbox_2024_val_bestmodel_brtkore_compatible.pth
